## Loading Libraries

In [1]:
import pandas as pd
import numpy as np

import os
import pathlib

import torch
import torch.nn as nn
import glob as glob

from torch.utils.data import DataLoader
from torch.optim import Adam
#from torch.autograd import Variable

import torchvision
from torchvision.transforms import transforms

In [2]:
## Checking for device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Transforms

In [3]:
transformer = transforms.Compose([
    # resize all the images to 150*150
    transforms.Resize((150,150)),
    
    # This has a 50% probability of the image is either original or horizontali flip
    transforms.RandomHorizontalFlip(), # so it will end up increasing the images by the factor of 2
    
    # change the color pixel in the range of 0-255 to 0-1, also change the data type from numpy to tensor
    transforms.ToTensor(),
    
    # transforms the range from 0-1 to -1-1, and in 2*3 matrix row= RGB Channel and Column= mean & Standard Deviation
    transforms.Normalize([0.5, 0.5, 0.5], # new pixels = (X - mean)/ Std
                        [0.5, 0.5, 0.5])
])

## Data Loader

### Path for Testing and Training

In [20]:
train_path = "/home/Users/anshulchaurasia/Downloads/HackerEarth/Intel Image Classification/seg_train"
test_path = "/home/Users/anshulchaurasia/Downloads/HackerEarth/Intel Image Classification/seg_test"

<h3>Note:</h3> When we have the lage number of images the we have to load them in <b>batches</b>. Because when we will trying loading all the images at once we will endup with <b>O</b>ut <b>O</b>f <b>M</b>emory error.

In [5]:
train_loader = DataLoader(torchvision.datasets.ImageFolder(train_path, transform= transformer),
                         batch_size = 256, # higher the batch size can lead the memory error
                         shuffle = True # will remove the model bias in case the order of images will be same
                         )

test_loader = DataLoader(torchvision.datasets.ImageFolder(test_path, transform= transformer),
                         batch_size = 256, # higher the batch size can lead the memory error
                         shuffle = True # will remove the model bias in case the order of images will be same
                        )

### categories

In [6]:
# to fetch all the categories, names 
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


## CNN Network

In [14]:
class ConvNet(nn.Module):
    ######################### Defining the constructor to extend the module class #####################
    def __init__(self, num_classes =6):
        super(ConvNet, self).__init__()
        
        # Output size after convolution filter
        #((w - f + 2p) / s) + 1
        
        # In this we will specify all the layers in our networks
        
        #########################[Start] Adding first layer #####################
        # Input shape = (batch size, #channels RGB, hight, width) i.e. (256,3,150,150)
        self.convl1 = nn.Conv2d(in_channels= 3, out_channels= 12, kernel_size= 3, stride= 1, padding= 1)
        # Now we have output channel is 12 then our so the shape changed to (256,12,150,150)
        
        # adding the batch Normalization
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Note: after the batch normalization shape will be same. i.e. (256,12,150,150)
        
        # adding the ReLu to bring the non linearity
        self.relu1 = nn.ReLU()
        # shape will be same (256,12,150,150)

        # adding the maxpool, which will decrease the hight and width of Convolution by the factor of 2
        self.pool = nn.MaxPool2d(kernel_size= 2)
        # shape (256, 12, 75, 75)
        #########################[End] Adding first layer #####################
        
        #########################[Start] Adding second layer #####################
        self.convl2 = nn.Conv2d(in_channels= 12, out_channels= 20, kernel_size= 3, stride= 1, padding= 1)
        # shape (256,20,75,75)
        self.relu2 = nn.ReLU()
        # shape (256,20,75,75)
        #########################[End] Adding second layer #####################
        
        #########################[Start] Adding second layer #####################
        self.convl3 = nn.Conv2d(in_channels= 20, out_channels= 32, kernel_size= 3, stride= 1, padding= 1)
        # shape (256,32,75,75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # shape (256,32,75,75)
        self.relu3 = nn.ReLU()
        # shape (256,32,75,75)
        #########################[End] Adding second layer #####################
        
        # Now we will add the fully connected layers
        self.fc = nn.Linear(in_features= 32*75*75, out_features= num_classes)
        
    ######################### Defining the constructor to extend the module class #####################
    
    ################################## Defining the forward function ##################################
    
    def forward(self,input):
        # passing all the inputs to the above layers
        output = self.convl1()
        output = self.bn1()
        output = self.relu1()
        output = self.pool()
        
        output = self.convl2()
        output = self.relu2()
        
        output = self.convl3()
        output = self.bn3()
        output = self.relu3()
        
        #Note: above output will be in a shape of matrix form, shape (256,32,75,75)
        
        # This will change the view 
        output = output.view(-1, 32*75*75)
        
        # post that we will feed it inside the fully connected layers
        output = output.fc(output)
        
        # return the final output
        return output
    
    ################################## Defining the forward function ##################################

In [15]:
# calling the ConvNet class
model = ConvNet(num_classes= 6).to(device)

### Optimizer and Loss Function

In [16]:
optimizer = Adam(model.parameters(), lr= 0.001, weight_decay= 0.001)
loss_function = nn.CrossEntropyLoss()

In [17]:
# defining hyperparameter 
num_epochs = 10

In [18]:
# calculating the size of training and testing images
train_count = len(glob.glob(train_path+'/**/*.jpg'))
test_count = len(glob.glob(test_path+'/**/*.jpg'))

print(train_count,test_count)

14034 3000


## Training the Model

In [19]:
best_accuracy = 0.0

for epoch in range(num_epochs):
    # Evaluation and Training on training dataset
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    # initialization of batches inside the training
    for i, (images, labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda)
            labels = Variable(labels.cuda)
        
        optimizer.zero_grad()
        
        # This will give us a prediction
        outputs = model(images)
        
        # Calculating errors
        loss = loss_function(outputs, labels)
        
        # this will do a back propagation
        loss.backward()
        
        # this will update weight and bias using computed gradients
        optimizer.step()
        
        train_loss += loss.cpu().data*images.size(0)
        _, prediction = torch.max(outputs.data,1)
        
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    
    # Evaluation on testing dataset
    model.eval()
    
    test_accuracy = 0.0
    
    # initialization of batches inside the training
    for i, (images, labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images = Variable(images.cuda)
            labels = Variable(labels.cuda)
            
        outputs += model(images)
        _,prediction = torch.max(outputs.data,1)
        
        test_accuracy += int(torch.sum(prediction == labels.data))
        
    test_accuracy = test_accuracy/test_count
    
    print("Epoch: "+str(epoch)+" Train Loss: "+str(int(train_loss))+" Training Accuracy: "+str(train_accuracy)+" Test Accuracy: "+str(test_accuracy))
    
    # saving the best model
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(),"best_checkpoint.model")
        best_accuracy = test_accuracy
        

TypeError: forward() missing 1 required positional argument: 'input'